In [ ]:
import os
import sys
import math
import pandas as pd
import random
import numpy as np
from tsfresh.feature_extraction import feature_calculators as fc
from scipy.signal import welch
from scipy.stats import linregress
LABELS = [1, 2, 3, 4, 5]
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
filename = './data.csv'

df = pd.read_csv(filename, index_col='Unnamed: 0')

In [ ]:
NUM_FOLDS = 5

# Extract the unique subjects from all the subject time series chunks.
subject_ids = set()
for sub in df.index.tolist():
    subject_ids.add('.'.join(sub.split('.')[1:]))   

# Organize those unique subjects by their time series' labels (all chunk within a
# subject have the same label, so we just look at the first chunk).
subjects_sorted = [[] for _ in LABELS]
for sub_id in subject_ids:
    label = df.loc['X1.{}'.format(sub_id), 'y'].item()
    subjects_sorted[label-1].append(sub_id)

# Randomly divide the subjects into the folds, and compile those folds so that they
# have an equal number of subject with each label in each fold.
subject_folds = [[] for _ in range(NUM_FOLDS)]
for subjects_label_group in subjects_sorted:
    # Partition by walking through the shuffled list with steps of size NUM_FOLDS
    random.shuffle(subjects_label_group)
    subject_partitions = [subjects_label_group[i::NUM_FOLDS] for i in range(NUM_FOLDS)]
    for j in range(NUM_FOLDS): 
        subject_folds[j] += subject_partitions[j]

In [ ]:
# Take a time_series and return its features
def create_features(chunk):
    feature_vector = []
    
    feature_vector.append(fc.mean(chunk))
    feature_vector.append(fc.variance(chunk))
    feature_vector.append(fc.minimum(chunk))
    feature_vector.append(fc.median(chunk))
    feature_vector.append(fc.maximum(chunk))
    
    # Welch's method to measure the density of rapid changes in signals
    feature_vector.append(fc.mean(welch(chunk, nperseg=178)))
    
    # Number of times two consecutive readings are on opposite sides of the mean
    feature_vector.append(fc.number_crossing_m(chunk, fc.mean(chunk)))
    
    # Slopes for each half of the time series
    slope, _, _, _, _ = linregress(x=range(89), y=chunk[:89])
    feature_vector.append(slope)
    slope, _, _, _, _ = linregress(x=range(89), y=chunk[89:])
    feature_vector.append(slope)
    
    return feature_vector

In [ ]:
# Cross validate
for fold_num in range(NUM_FOLDS):
    
    # Split up the chunks into training and testing sets per the current testing fold
    train_chunks = []
    train_labels = []
    test_chunks = []
    test_labels = []
    for i in range(NUM_FOLDS):
        if fold_num == i:
            for sub_id in subject_folds[i]:
                for j in range(23):
                    chunk_id = 'X{}.{}'.format(j+1, sub_id)
                    test_chunks.append(create_features(df.loc[chunk_id, 'X1':'X178'].values))
                    test_labels.append(1 if df.loc[chunk_id, 'y'].item() == 1 else 0)
        else:
            for sub_id in subject_folds[i]:
                for j in range(23):
                    chunk_id = 'X{}.{}'.format(j+1, sub_id)
                    train_chunks.append(create_features(df.loc[chunk_id, 'X1':'X178'].values))
                    train_labels.append(1 if df.loc[chunk_id, 'y'].item() == 1 else 0)
    
    # Train and test on LogisticRegression and ExtraTrees
    log_reg = LogisticRegression()
    log_reg.fit(train_chunks, train_labels)
    extra_trees = ExtraTreesClassifier()
    extra_trees.fit(train_chunks, train_labels)
    
    # Run on held out data
    successes_seizure_log = 0
    successes_non_seizure_log = 0
    errors_seizure_log = 0
    errors_non_seizure_log = 0
    successes_seizure_trees = 0
    successes_non_seizure_trees = 0
    errors_seizure_trees = 0
    errors_non_seizure_trees = 0
    for chunk, label in zip(test_chunks, test_labels):
        if log_reg.predict(np.asarray(chunk).reshape(1, -1)) == label:
            if label == 1:
                successes_seizure_log += 1
            else: 
                successes_non_seizure_log += 1
        else: 
            if label == 1:
                errors_seizure_log += 1
            else: 
                errors_non_seizure_log += 1
                
        if extra_trees.predict(np.asarray(chunk).reshape(1, -1)) == label:
            if label == 1:
                successes_seizure_trees += 1
            else: 
                successes_non_seizure_trees += 1
        else: 
            if label == 1:
                errors_seizure_trees += 1
            else: 
                errors_non_seizure_trees += 1
    print('Fold {}'.format(fold_num))
    print(' Log Accuracy: \t\t{}/{}: {}'
          .format(successes_seizure_log + successes_non_seizure_log, 
                  len(test_chunks),
                  round((successes_seizure_log + successes_non_seizure_log) 
                        / len(test_chunks), 4)))
    print(' Predicted Seizure Correctly: {}'.format(successes_seizure_log))
    print(' Predicted Seizure Incorrectly: {}'.format(errors_non_seizure_log))
    print(' Predicted Non-Seizure Correctly: {}'.format(successes_non_seizure_log))
    print(' Predicted Non-Seizure Incorrectly: {}'.format(errors_seizure_log))
    
    print(' Trees Accuracy: \t{}/{}: {}'
          .format(successes_seizure_trees + successes_non_seizure_trees, 
                  len(test_chunks),
                  round((successes_seizure_trees + successes_non_seizure_trees) 
                        / len(test_chunks), 4)))
    print(' Predicted Seizure Correctly: {}'.format(successes_seizure_trees))
    print(' Predicted Seizure Incorrectly: {}'.format(errors_non_seizure_trees))
    print(' Predicted Non-Seizure Correctly: {}'.format(successes_non_seizure_trees))
    print(' Predicted Non-Seizure Incorrectly: {}'.format(errors_seizure_trees))
    
    # Training loss
    errors_log = 0
    errors_trees = 0
    for chunk, label in zip(train_chunks, train_labels):
        if log_reg.predict(np.asarray(chunk).reshape(1, -1)) != label:
            errors_log += 1
        if extra_trees.predict(np.asarray(chunk).reshape(1, -1)) != label:
            errors_trees += 1
    print(' Log Training Loss:\t{}/{}: {}\n Trees Training Loss:\t{}/{}: {} \n'
          .format(errors_log, len(train_chunks),
                  round(errors_log / len(train_chunks), 4),
                  errors_trees, len(train_chunks),
                  round(errors_trees / len(train_chunks), 4))) 
    